### EDA для NLP ML-модели предсказания уровня зарплаты

### Импортируем библиотеки

In [1]:
# импортируем библиотеки
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from matplotlib import pyplot as plt
import seaborn as sns 
import matplotlib

In [2]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

In [3]:
#настройка, чтобы можно было просматривать все столбцы датафрейма
matplotlib.rcParams.update({'font.size': 14})
pd.options.display.max_columns = 100

In [4]:
# этот волшебный код убирает ненужные предупреждения об ошибках
import warnings
warnings.filterwarnings('ignore')

### Загружаем данные

In [5]:
# # адреса для рабочего компа
adres_train = 'C:/Users/lazarevnv/Desktop/vacancy/train.csv'
# adres_train = 'C:/Users/lazarevnv/OneDrive/!Обучение/RuCode/vacancy_salary_pred/'
adres_test = 'C:/Users/lazarevnv/OneDrive/1_Обучение/RuCode/vacancy_salary_pred/test.csv'
adres_sample_submission = 'C:/Users/lazarevnv/OneDrive/1_Обучение/RuCode/vacancy_salary_pred/sample_submission.csv'

In [5]:
# адреса для домашнего компа
adres_train = 'C:/Users/Ник/Desktop/ml/RuCode5_vacancy_salary_pred/train.csv'
adres_test = 'C:/Users/Ник/OneDrive/1_Обучение/RuCode/vacancy_salary_pred/test.csv'
adres_sample_submission = 'C:/Users/Ник/OneDrive/1_Обучение/RuCode/vacancy_salary_pred/sample_submission.csv'

In [6]:
# df_train = pd.read_csv(adres_train, index_col = 'id')
df_train = pd.read_csv(adres_train)

### Готовим отдельный df для NLP

In [10]:
df_nlp = df_train[['additional_info', 'career_perspective',
         'job_benefits', 'job_benefits_other_benefits', 
         'job_location_address', 'metro_station', 
         'requirements_qualifications', 'requirements_required_certificates', 
         'responsibilities', 'retraining_condition', 
         'social_protecteds_social_protected', 'title']].head(10)

In [11]:
df_nlp.tail(3)

,additional_info,career_perspective,job_benefits,job_benefits_other_benefits,job_location_address,metro_station,requirements_qualifications,requirements_required_certificates,responsibilities,retraining_condition,social_protecteds_social_protected,title
7,Дисциплинированность / Ответственность,NaN,NaN,Социальный пакет / Предоставление спец.одежды,Кемеровская область,NaN,"Опыт работы по профессии от одного года, готов...",NaN,"сборка и монтаж металлоконструкций, соблюдение...",NaN,NaN,Слесарь по сборке металлоконструкций 1 разряда...
8,Квотируемое рабочее место / Длительные команди...,NaN,NaN,Социальный пакет,Красноярский край,NaN,Знание ГОСТов по части хранения опасных грузов...,NaN,"Осуществляет прием на склад, хранение и выдачу...",NaN,Инвалиды,Кладовщик
9,стажировка выпускников / Коммуникабельность / ...,NaN,NaN,Социальный пакет / Работа в динамично развиваю...,Тюменская область,NaN,"Умение работать с возражениями и претензиями, ...",NaN,Разрабатывать или принимать участие в разработ...,NaN,NaN,Юрист


### Работаем с текстовым полем

In [12]:
import nltk
import re

In [13]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ник\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df_nlp['additional_info'] = df_nlp['additional_info'].fillna('пусто')

In [15]:
df_nlp['additional_info']

0    Производство строительных металлических констр...
1    Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...
2    Новое рабочее место / Наружные трубопроводы и ...
3    Сохраненное рабочее место / Дисциплинированнос...
4                         Дополнительное рабочее место
5                                                пусто
6    контрактный управляющий (специалист в сфере за...
7               Дисциплинированность / Ответственность
8    Квотируемое рабочее место / Длительные команди...
9    стажировка выпускников / Коммуникабельность / ...
Name: additional_info, dtype: object

In [16]:
# Токенизация:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [17]:
# разделили на токены
df_nlp['additional_token'] = df_nlp.apply(lambda row: tokenizer.tokenize(row['additional_info'].lower()), axis = 1)

In [18]:
df_nlp[['additional_info','additional_token']].head()

,additional_info,additional_token
0,Производство строительных металлических констр...,"[производство, строительных, металлических, ко..."
1,Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...,"[новое, рабочее, место, водитель, автобуса, ка..."
2,Новое рабочее место / Наружные трубопроводы и ...,"[новое, рабочее, место, наружные, трубопроводы..."
3,Сохраненное рабочее место / Дисциплинированнос...,"[сохраненное, рабочее, место, дисциплинированн..."
4,Дополнительное рабочее место,"[дополнительное, рабочее, место]"


### Проведем Лемматизацию, получим нормальную (начальную) форму слова

In [19]:
import re

In [20]:
from pymorphy2 import MorphAnalyzer

In [21]:
from nltk.corpus import stopwords

In [22]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

In [23]:
def lemmatize(doc):
    doc = re.sub(patterns, ' ', str(doc)) #?
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    return tokens

In [24]:
# новое поле лемматизация
df_nlp['additional_lemm'] = df_nlp['additional_token'].apply(lemmatize)

In [26]:
df_nlp[['additional_info','additional_token', 
        'additional_lemm']]

,additional_info,additional_token,additional_lemm
0,Производство строительных металлических констр...,"[производство, строительных, металлических, ко...","[производство, строительный, металлический, ко..."
1,Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...,"[новое, рабочее, место, водитель, автобуса, ка...","[новый, рабочий, место, водитель, автобус, кат..."
2,Новое рабочее место / Наружные трубопроводы и ...,"[новое, рабочее, место, наружные, трубопроводы...","[новый, рабочий, место, наружное, трубопровод,..."
3,Сохраненное рабочее место / Дисциплинированнос...,"[сохраненное, рабочее, место, дисциплинированн...","[сохранить, рабочий, место, дисциплинированнос..."
4,Дополнительное рабочее место,"[дополнительное, рабочее, место]","[дополнительный, рабочий, место]"
5,пусто,[пусто],[пусто]
6,контрактный управляющий (специалист в сфере за...,"[контрактный, управляющий, специалист, в, сфер...","[контрактный, управлять, специалист, сфера, за..."
7,Дисциплинированность / Ответственность,"[дисциплинированность, ответственность]","[дисциплинированность, ответственность]"
8,Квотируемое рабочее место / Длительные команди...,"[квотируемое, рабочее, место, длительные, кома...","[квотируемый, рабочий, место, длительный, кома..."
9,стажировка выпускников / Коммуникабельность / ...,"[стажировка, выпускников, коммуникабельность, ...","[стажировка, выпускник, коммуникабельность, ра..."


### Убираем стоп слова

In [27]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ник\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
stopWords_ru = set(stopwords.words('russian'))
print(stopWords_ru)

{'какой', 'он', 'ему', 'хоть', 'себе', 'чтоб', 'конечно', 'ей', 'об', 'ней', 'всегда', 'за', 'из', 'тоже', 'было', 'ты', 'один', 'нее', 'они', 'без', 'может', 'моя', 'такой', 'тем', 'можно', 'как', 'мой', 'ее', 'там', 'при', 'не', 'где', 'больше', 'над', 'уж', 'к', 'мне', 'о', 'вдруг', 'него', 'сейчас', 'между', 'их', 'с', 'эти', 'когда', 'разве', 'от', 'вас', 'потом', 'для', 'ним', 'три', 'еще', 'со', 'только', 'есть', 'был', 'я', 'до', 'тогда', 'даже', 'и', 'или', 'ж', 'под', 'тут', 'но', 'вы', 'уже', 'нибудь', 'этот', 'были', 'наконец', 'кто', 'на', 'у', 'здесь', 'впрочем', 'свою', 'потому', 'про', 'его', 'того', 'чего', 'а', 'так', 'куда', 'никогда', 'теперь', 'эту', 'всего', 'всех', 'этой', 'нас', 'всю', 'чтобы', 'лучше', 'зачем', 'быть', 'чуть', 'нет', 'том', 'во', 'что', 'то', 'если', 'мы', 'в', 'все', 'этом', 'хорошо', 'меня', 'им', 'вот', 'она', 'ли', 'опять', 'будто', 'чем', 'тот', 'ну', 'будет', 'ведь', 'них', 'бы', 'нельзя', 'перед', 'раз', 'ничего', 'совсем', 'более', 'мно

In [29]:
# новое поле без стоп слов
df_nlp['additional_stop'] = df_nlp.apply(lambda row: \
[word for word in row['additional_token'] if word not in stopWords_ru], axis = 1)

In [30]:
df_nlp[['additional_info','additional_token', 'additional_stop'
        , 'additional_lemm'
#         , 'additional_bigramms'
       ]].head(3)

,additional_info,additional_token,additional_stop,additional_lemm
0,Производство строительных металлических констр...,"[производство, строительных, металлических, ко...","[производство, строительных, металлических, ко...","[производство, строительный, металлический, ко..."
1,Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...,"[новое, рабочее, место, водитель, автобуса, ка...","[новое, рабочее, место, водитель, автобуса, ка...","[новый, рабочий, место, водитель, автобус, кат..."
2,Новое рабочее место / Наружные трубопроводы и ...,"[новое, рабочее, место, наружные, трубопроводы...","[новое, рабочее, место, наружные, трубопроводы...","[новый, рабочий, место, наружное, трубопровод,..."


### Обучение модели Word2Vec

In [31]:
from gensim.models import Word2Vec

Модель имеет множество аргументов:

min_count — игнорировать все слова с частотой встречаемости меньше, чем это значение.

windоw — размер контекстного окна, о котором говорили тут, обозначает диапазон контекста.

size — размер векторного представления слова (word embedding).

negative — сколько неконтекстных слов учитывать в обучении, используя negative sampling, о нем также упоминалось здесь.

alpha — начальный learning_rate, используемый в алгоритме обратного распространения ошибки (Backpropogation).

min_alpha — минимальное значение learning_rate, на которое может опуститься в процессе обучения.

sg — если 1, то используется реализация Skip-gram; если 0, то CBOW. О реализациях также говорили тут.

In [32]:
w2v_model = Word2Vec(
    min_count=1, # 10 для полного поля
    window=2,
    size=300,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)

In [33]:
data = df_nlp['additional_lemm']

In [34]:
# получить словарь
w2v_model.build_vocab(data)

In [35]:
# обучить модель, используя метод train
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(106, 2100)

In [ ]:
# Если в дальнейшем не требуется снова обучать модель, то для сохранения оперативной памяти можно написать следующее:
# w2v_model.init_sims(replace=True)

In [36]:
# переводим в общий вектор список обработанных слов в каждой строке
df_nlp['additional_vec'] = [(sum(w2v_model.wv[df_nlp['additional_lemm']\
    [i]])/len(df_nlp['additional_lemm'][i])) for i in range(0, len(df_nlp['additional_lemm']))]

### Усредняем вектор

In [44]:
df_nlp['additional_midle'] = [sum(df_nlp['additional_vec'][i])/len(df_nlp['additional_vec']\
    [i]) for i in range(0, len(df_nlp['additional_vec']))]

In [45]:
df_nlp['additional_midle']

0    5.679543e-06
1    1.405483e-05
2    3.025096e-07
3    2.050570e-05
4    2.102180e-05
5   -1.996602e-05
6    2.119539e-06
7    3.515241e-05
8    1.668924e-05
9   -1.475015e-05
Name: additional_midle, dtype: float64

In [46]:
df_nlp.head(3)

,additional_info,career_perspective,job_benefits,job_benefits_other_benefits,job_location_address,metro_station,requirements_qualifications,requirements_required_certificates,responsibilities,retraining_condition,social_protecteds_social_protected,title,additional_token,additional_lemm,additional_stop,additional_vec,additional_midle
0,Производство строительных металлических констр...,NaN,NaN,Социальный пакет / социальный пакет,Московская область,NaN,"Должен знать: постановления, распоряжения, при...",NaN,"Разрабатывает эскизные, технические и рабочие ...",NaN,NaN,Инженер-конструктор,"[производство, строительных, металлических, ко...","[производство, строительный, металлический, ко...","[производство, строительных, металлических, ко...","[-0.00054909254, -0.00019573965, -0.0005334590...",5.679543e-06
1,Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...,NaN,NaN,Социальный пакет / кат.Д,Республика Башкортостан,NaN,"Управлять автомобилем, соблюдать правила повед...",NaN,"Профессиональное вождение автомобиля, обеспечи...",NaN,NaN,Водитель автомобиля,"[новое, рабочее, место, водитель, автобуса, ка...","[новый, рабочий, место, водитель, автобус, кат...","[новое, рабочее, место, водитель, автобуса, ка...","[-0.0003235098, -0.000270087, -9.630842e-05, -...",1.405483e-05
2,Новое рабочее место / Наружные трубопроводы и ...,NaN,NaN,Предоставление жилья / Социальный пакет,Республика Башкортостан,NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,NaN,Монтажник,"[новое, рабочее, место, наружные, трубопроводы...","[новый, рабочий, место, наружное, трубопровод,...","[новое, рабочее, место, наружные, трубопроводы...","[0.00014113513, -7.436288e-05, 7.813615e-05, 3...",3.025096e-07


### Метод получения данных для нового векторного поля

In [47]:
'''Токенизация'''
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

'''Лемматизация'''
import re
from pymorphy2 import MorphAnalyzer

'''Стоп-слова'''
from nltk.corpus import stopwords
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

'''Обучение модели Word2Vec'''
from gensim.models import Word2Vec



In [48]:
def lemmatize(doc):
    '''Лемматизация, получим нормальную (начальную) форму слова'''
    #doc - это содержимое в каждой ячейке, предложение, которое нужно изменить
    doc = re.sub(patterns, ' ', str(doc))
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)

    return tokens

In [49]:
def vector_row(df_nlp, row_name):
    '''Метод получения данных для векторизации текстового поля'''
    
    '''Заполним пустые строки'''
    df_nlp[row_name] = df_nlp[row_name].fillna('пусто')
    
    '''Получим токены текстового поля'''
    new_name = row_name + '_vec'
    df_nlp[new_name] = df_nlp.apply\
        (lambda row: tokenizer.tokenize\
        (row[row_name].lower()), axis = 1)
        
    '''Лемматизация, получим нормальную (начальную) форму слова 
    (именительный падеж)'''
    df_nlp[new_name] = df_nlp[new_name].apply(lemmatize)
    
    '''Убираем стоп-слова'''
    df_nlp[new_name] = df_nlp.apply(lambda row: \
        [word for word in row[new_name] if word \
        not in stopWords_ru], axis = 1)
    
    '''Обучение модели'''
    # зададим параметры модели
    w2v_model = Word2Vec(
        min_count=1, # 10 для полного поля, для реальной работы
        window=2,
        size=300,
        negative=10,
        alpha=0.03,
        min_alpha=0.0007,
        sample=6e-5,
        sg=1)
    
    # получить словарь
    w2v_model.build_vocab(df_nlp[new_name])
    # обучить модель, используя метод train
    w2v_model.train(data, total_examples = w2v_model.corpus_count,\
                    epochs=30, report_delay=1)
    
    '''переводим в общий вектор список обработанных слов в каждой строке'''
    df_nlp[new_name] = \
        [(sum(w2v_model.wv[df_nlp[new_name][i]])\
          /len(df_nlp[new_name][i])) \
         for i in range(0, len(df_nlp[new_name]))]
    
    '''Вычисляем среднее значение для вектора'''   
    df_nlp[new_name] = \
        [sum(df_nlp[new_name][i])
         /len(df_nlp[new_name][i])
         for i in range(0, len(df_nlp[new_name]))]  

    return

In [50]:
def text_rows_nlp_vector(df_nlp, text_rows):
    for row in text_rows:
        vector_row(df_nlp, row)
        df_nlp.drop(row, axis = 1, inplace = True) 
    return

In [51]:
# текстовый поля, пока так. Но потом можно сделать по другому, После обработки других полей.
text_rows = ['additional_info', 'career_perspective',
         'job_benefits', 'job_benefits_other_benefits', 
         'job_location_address', 'metro_station', 
         'requirements_qualifications', 'requirements_required_certificates', 
         'responsibilities', 'retraining_condition', 
         'social_protecteds_social_protected', 'title']

In [209]:
df = df_nlp.head(3)

In [215]:
# проверка применения метода vector_row
row_name = 'additional_info'
vector_row(df_nlp, row_name)

In [223]:
# проверка применения метода vector_row
row_name = 'additional_info'
vector_row(df, row_name)

In [227]:
# проверка применения метода text_rows_nlp_vector
text_rows_nlp_vector(df_nlp, text_rows)

In [225]:
# проверка применения метода text_rows_nlp_vector
text_rows_nlp_vector(df, text_rows)

In [226]:
df

,additional_info_vec,career_perspective_vec,job_benefits_vec,job_benefits_other_benefits_vec,job_location_address_vec,metro_station_vec,requirements_qualifications_vec,requirements_required_certificates_vec,responsibilities_vec,retraining_condition_vec,social_protecteds_social_protected_vec,title_vec
id,,,,,,,,,,,,
0,0.000007,0.000051,0.000051,0.000061,0.000010,0.000051,0.000012,0.000051,0.000002,0.000051,0.000051,0.000042
1,0.000022,0.000051,0.000051,0.000050,-0.000030,0.000051,-0.000009,0.000051,-0.000002,0.000051,0.000051,-0.000005
2,0.000027,0.000051,0.000051,0.000006,-0.000030,0.000051,0.000016,0.000051,0.000016,0.000051,0.000051,0.000049
3,0.000030,0.000051,0.000051,0.000061,0.000006,0.000051,-0.000009,0.000051,0.000004,0.000051,0.000051,-0.000005
4,0.000030,0.000051,0.000051,0.000061,-0.000009,0.000051,0.000001,0.000051,-0.000005,0.000051,0.000051,0.000057


In [228]:
df_nlp.head(3)

,additional_info_vec,career_perspective_vec,job_benefits_vec,job_benefits_other_benefits_vec,job_location_address_vec,metro_station_vec,requirements_qualifications_vec,requirements_required_certificates_vec,responsibilities_vec,retraining_condition_vec,social_protecteds_social_protected_vec,title_vec
id,,,,,,,,,,,,
0,0.000007,0.000051,0.000051,0.000061,0.00001,0.000051,0.000012,0.000051,0.000002,0.000051,0.000051,0.000042
1,0.000022,0.000051,0.000051,0.000050,-0.00003,0.000051,-0.000009,0.000051,-0.000002,0.000051,0.000051,-0.000005
2,0.000027,0.000051,0.000051,0.000006,-0.00003,0.000051,0.000016,0.000051,0.000016,0.000051,0.000051,0.000049


### Метод обработки категориальных полей

In [52]:
def row_worker(df, rows_list, worker_funk):
    '''Метод применения метода для перечня полей'''
    for row in rows_list: # цикл для всех полей
        worker_funk(df, row)
    return

In [53]:
def category_change(df, row):
    '''Метод обработки категориальных данных'''
    # подготовим список категорий
    category_list = list(df[row].unique())
    # убираем из списка nan
    category_list = [x for x in category_list if x == x]

    count = 1 #счетчик
    for category in category_list:
        df.loc[df[row] == category, row] = count
        count += 1
    return

In [58]:
# проверка работы метода category_row
category_rows = ['accommodation_housing', 'education_requirements_education_type']

In [59]:
df_tr = df_train.loc[(df_train['accommodation_housing'] == 'DORMITORY') \
             | (df_train['accommodation_housing'] == 'ROOM')\
             | (df_train['accommodation_housing'] == 'FLAT')].head(5)

In [61]:
category_rows_list = ['accommodation_housing', 'education_requirements_education_type']

In [62]:
#  применение метода category_change через метод работы с полями
row_worker(df_tr, category_rows_list, category_change)

In [63]:
df_tr[['accommodation_housing', 'education_requirements_education_type']]

,accommodation_housing,education_requirements_education_type
16,1,1
89,2,2
111,2,NaN
112,2,2
115,2,3


### Метод работы с датами

In [54]:
import datetime

In [71]:
# df_tr[['date_creation', 'date_posted']]

In [55]:
def date_row(df, row):
    '''Метод работы с датами'''
    df[row + '_year'] = pd.DatetimeIndex(df_tr[row]).year
    df[row + '_month'] = pd.DatetimeIndex(df_tr[row]).month
    df.drop(row, axis = 1, inplace = True)
    return

In [56]:
date_rows = ['date_creation', 'date_posted']

In [64]:
row_worker(df_tr, date_rows, date_row)

In [65]:
df_tr.head(2)

,id,accommodation_capability,accommodation_housing,additional_info,base_salary,base_salary_max,base_salary_min,career_perspective,caring_workers,date_creation_mistake,date_posted_mistake,date_time_posted,disabled,dms,drive_licences,driver_licence_a,driver_licence_b,driver_licence_c,driver_licence_d,driver_licence_e,education_academic_degree,education_requirements_education_type,education_requirements_speciality,employment_type,experience_requirements,federal_district,id_hiring_organization,incentive_compensation_transport_compensation,industry,inner_info_contact_source,inner_info_source_type,inner_info_status,is_uzbekistan_recruitment,job_benefits,job_benefits_other_benefits,job_location_additional_address_info,job_location_address,job_location_geo_latitude,job_location_geo_longitude,large_families,metro_station,minor_workers,need_medcard,okso_code,organization,payment_meals,payment_sports_activities,premium_size,premium_type,profession,region,released_persons,requirements_id_priority_category,requirements_qualifications,requirements_required_certificates,responsibilities,retraining_condition,retraining_grant,retraining_grant_value,single_parent,social_protecteds_social_protected,source,time_posted,title,vouchers_health_institutions,work_hours,work_places,workers_with_disabled_children,mean_salary,date_creation_year,date_creation_month,date_posted_year,date_posted_month
16,16,1.0,1,Сохраненное рабочее местоОтветственность,от 29721 до 45880,45880.0,29721.0,NaN,0.0,NaN,NaN,2019-02-20 11:37:41,0,0.0,NaN,0,0,0,0,0,NaN,1,NaN,Полная занятость,NaN,NaN,1022200562450,NaN,Medicine,COMPANY,Вакансия работодателя,Одобрено,NaN,NaN,Выплата подъемных в сумме 250 000 руб. Служебн...,"659315, г Бийск, ул Ленинградская, д. 94","Алтайский край, г. Бийск, Ленинградская улица",52.502863,85.140454,0,NaN,0.0,NaN,NaN,1022200562450,0,0.0,NaN,NaN,204751.0,2200000000000,NaN,NaN,"Действующий сертификат ""Терапия"". Наличие сани...",сертификат по специальности,Работать в терапевтическом отделении поликлини...,NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,11:37:41,Врач-терапевт участковый,0,Полный рабочий день,4,0.0,37800.5,2018,12,2019,2
89,89,1.0,2,М.р.: ООО 'ДОРОЖНО-СТРОИТЕЛЬНЫЙ ТРЕСТ' №1 / Ма...,от 61000 до 61000,61000.0,61000.0,NaN,0.0,NaN,NaN,2018-05-09 11:30:23,0,0.0,NaN,0,0,0,0,0,NaN,2,NaN,Полная занятость,3.0,NaN,1028601679061,PASSAGE_PAID,BuldindRealty,COMPANY,Вакансия работодателя,Одобрено,NaN,NaN,Наличие общежития / Социальный пакет / предост...,"Тюменская область,г. Сургут,ул.Промышленная,д.8",Республика Башкортостан,61.285770,73.401513,0,NaN,0.0,NaN,NaN,1028601679061,0,0.0,NaN,NaN,NaN,200000000000,NaN,NaN,"свидетельство об обучении на автоэлектрика,доб...",NaN,обслуживание и ремонт автотранспорта Выполнени...,NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,11:30:23,Автоэлектрик,0,Вахтовый метод,1,0.0,61000.0,2017,12,2018,5


### Метод работы с неадекватными датами

In [490]:
def date_mistake(df, row_mistake, rows_correct):
    '''Метод работы с неадекватными датами, замена на среднее значение'''
    for row_cor in rows_correct:
        df[row_cor] = 0
        print(row_cor, '\n')
        print(df[[row_cor]])
#         df.loc[(df[row_mistake].isna()), row_cor] = round(df[row_cor].mean())
        df.loc[(df[row_mistake] == 1) & (df[row_cor].notna()), row_cor] = round(df[row_cor].mean())
    return
# df - датафрейм
# row_mistake - поле с данными об ошибке
# rows_correct - список полей, где нужно вносить корректировки

In [660]:
df_date_mistake = df_train.head(3).copy()


In [661]:
df_date_mistake['date_creation_year'] = 2019
df_date_mistake['date_creation_month'] = 12

In [678]:
#  проверка работы date_mistake
row_mistake = 'date_creation_mistake'
df_date_mistake['date_creation_mistake'] = 1
rows_correct = ['date_creation_year', 'date_creation_month']
# date_mistake(df_date_mistake[['date_creation_mistake']], row_mistake, rows_correct) # выдает ошибку

In [501]:
print(round(df_date_mistake['date_creation_year'].mean()))
print(round(df_date_mistake['date_creation_month'].mean()))

2019
12


In [679]:
df_date_mistake.loc[df_date_mistake[row_mistake] == 1]

,id,accommodation_capability,accommodation_housing,additional_info,base_salary,base_salary_max,base_salary_min,career_perspective,caring_workers,date_creation,date_creation_mistake,date_posted,date_posted_mistake,date_time_posted,disabled,dms,drive_licences,driver_licence_a,driver_licence_b,driver_licence_c,driver_licence_d,driver_licence_e,education_academic_degree,education_requirements_education_type,education_requirements_speciality,employment_type,experience_requirements,federal_district,id_hiring_organization,incentive_compensation_transport_compensation,industry,inner_info_contact_source,inner_info_source_type,inner_info_status,is_uzbekistan_recruitment,job_benefits,job_benefits_other_benefits,job_location_additional_address_info,job_location_address,job_location_geo_latitude,job_location_geo_longitude,large_families,metro_station,minor_workers,need_medcard,okso_code,organization,payment_meals,payment_sports_activities,premium_size,premium_type,profession,region,released_persons,requirements_id_priority_category,requirements_qualifications,requirements_required_certificates,responsibilities,retraining_condition,retraining_grant,retraining_grant_value,single_parent,social_protecteds_social_protected,source,time_posted,title,vouchers_health_institutions,work_hours,work_places,workers_with_disabled_children,mean_salary,date_creation_year,date_creation_month
0,0,NaN,NaN,Производство строительных металлических констр...,от 50000 до 50000,50000.0,50000.0,NaN,0.0,2019-07-10,1,2019-07-05,NaN,2019-07-05 00:00:00,0,0.0,NaN,0,0,0,0,0,NaN,Высшее,Металлообрабатывающие станки и комплексы,Полная занятость,1.0,NaN,1027739386938,NaN,Industry,COMPANY,Вакансия службы занятости,Одобрено,NaN,NaN,Социальный пакет / социальный пакет,"141281, Московская обл, Ивантеевка г, Санаторн...",Московская область,55.986148,37.923422,0,NaN,0.0,NaN,NaN,1027739386938,0,0.0,NaN,NaN,NaN,5000000000000,NaN,NaN,"Должен знать: постановления, распоряжения, при...",NaN,"Разрабатывает эскизные, технические и рабочие ...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Инженер-конструктор,0,Полный рабочий день,1,0.0,50000.0,2019,12
1,1,NaN,NaN,Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...,от 25000 до 25000,25000.0,25000.0,NaN,0.0,2019-12-27,1,2019-12-26,NaN,2019-12-26 00:00:00,0,0.0,[D],0,0,0,1,0,NaN,Среднее профессиональное,NaN,Полная занятость,1.0,NaN,1100265000890,NaN,Communal,COMPANY,Вакансия службы занятости,Одобрено,f,NaN,Социальный пакет / кат.Д,"452607, г Октябрьский, ул Кувыкина, д. 3, корп. а",Республика Башкортостан,54.486407,53.483033,0,NaN,0.0,NaN,NaN,1100265000890,0,0.0,NaN,NaN,114428.0,200000000000,NaN,NaN,"Управлять автомобилем, соблюдать правила повед...",NaN,"Профессиональное вождение автомобиля, обеспечи...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Водитель автомобиля,0,Полный рабочий день,1,0.0,25000.0,2019,12
2,2,1.0,NaN,Новое рабочее место / Наружные трубопроводы и ...,от 40000 до 40000,40000.0,40000.0,NaN,0.0,2020-08-22,1,2020-08-21,NaN,2020-08-21 00:00:00,0,0.0,NaN,0,0,0,0,0,NaN,Среднее профессиональное,NaN,Полная занятость,1.0,4.0,1180280071058,NaN,Industry,COMPANY,Служба занятости,Одобрено,f,NaN,Предоставление жилья / Социальный пакет,"450076, г Уфа, ул Пушкина, д. 35, офис 2",Республика Башкортостан,54.725753,55.925139,0,NaN,0.0,NaN,NaN,1180280071058,0,0.0,NaN,NaN,145445.0,200000000000,0.0,NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Монтажник,0,Полный рабочий день,5,0.0,40000.0,2019,12


In [682]:
for row_cor in rows_correct:
#         df[row_cor] = 0
    print(row_cor, '\n')
    print(df_date_mistake[[row_cor]], '\n')
#     df_date_mistake.loc[(df_date_mistake[row_mistake] == 1), row_cor] = round(df_date_mistake[row_cor].mean())
    print(df_date_mistake.loc[df_date_mistake[row_mistake] == 1])

date_creation_year 

   date_creation_year
0                2019
1                2019
2                2019 

   id  accommodation_capability accommodation_housing  \
0   0                       NaN                   NaN   
1   1                       NaN                   NaN   
2   2                       1.0                   NaN   

                                     additional_info        base_salary  \
0  Производство строительных металлических констр...  от 50000 до 50000   
1  Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...  от 25000 до 25000   
2  Новое рабочее место / Наружные трубопроводы и ...  от 40000 до 40000   

   base_salary_max  base_salary_min career_perspective  caring_workers  \
0          50000.0          50000.0                NaN             0.0   
1          25000.0          25000.0                NaN             0.0   
2          40000.0          40000.0                NaN             0.0   

  date_creation  date_creation_mistake date_posted  date_poste

2                2019                   12  


In [683]:
for row_cor in rows_correct:
#         df[row_cor] = 0
    print(row_cor, '\n')
    print(df_date_mistake[[row_cor]])
#         df.loc[(df[row_mistake].isna()), row_cor] = round(df[row_cor].mean())
    df_date_mistake.loc[(df_date_mistake[row_mistake] == 1), row_cor] = round(df_date_mistake[row_cor].mean())

date_creation_year 

   date_creation_year
0                2019
1                2019
2                2019
date_creation_month 

   date_creation_month
0                   12
1                   12
2                   12


In [684]:
df_date_mistake.loc[(df_date_mistake[row_mistake] == 1) & (df_date_mistake[row_cor].notna())]

,id,accommodation_capability,accommodation_housing,additional_info,base_salary,base_salary_max,base_salary_min,career_perspective,caring_workers,date_creation,date_creation_mistake,date_posted,date_posted_mistake,date_time_posted,disabled,dms,drive_licences,driver_licence_a,driver_licence_b,driver_licence_c,driver_licence_d,driver_licence_e,education_academic_degree,education_requirements_education_type,education_requirements_speciality,employment_type,experience_requirements,federal_district,id_hiring_organization,incentive_compensation_transport_compensation,industry,inner_info_contact_source,inner_info_source_type,inner_info_status,is_uzbekistan_recruitment,job_benefits,job_benefits_other_benefits,job_location_additional_address_info,job_location_address,job_location_geo_latitude,job_location_geo_longitude,large_families,metro_station,minor_workers,need_medcard,okso_code,organization,payment_meals,payment_sports_activities,premium_size,premium_type,profession,region,released_persons,requirements_id_priority_category,requirements_qualifications,requirements_required_certificates,responsibilities,retraining_condition,retraining_grant,retraining_grant_value,single_parent,social_protecteds_social_protected,source,time_posted,title,vouchers_health_institutions,work_hours,work_places,workers_with_disabled_children,mean_salary,date_creation_year,date_creation_month
0,0,NaN,NaN,Производство строительных металлических констр...,от 50000 до 50000,50000.0,50000.0,NaN,0.0,2019-07-10,1,2019-07-05,NaN,2019-07-05 00:00:00,0,0.0,NaN,0,0,0,0,0,NaN,Высшее,Металлообрабатывающие станки и комплексы,Полная занятость,1.0,NaN,1027739386938,NaN,Industry,COMPANY,Вакансия службы занятости,Одобрено,NaN,NaN,Социальный пакет / социальный пакет,"141281, Московская обл, Ивантеевка г, Санаторн...",Московская область,55.986148,37.923422,0,NaN,0.0,NaN,NaN,1027739386938,0,0.0,NaN,NaN,NaN,5000000000000,NaN,NaN,"Должен знать: постановления, распоряжения, при...",NaN,"Разрабатывает эскизные, технические и рабочие ...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Инженер-конструктор,0,Полный рабочий день,1,0.0,50000.0,2019,12
1,1,NaN,NaN,Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...,от 25000 до 25000,25000.0,25000.0,NaN,0.0,2019-12-27,1,2019-12-26,NaN,2019-12-26 00:00:00,0,0.0,[D],0,0,0,1,0,NaN,Среднее профессиональное,NaN,Полная занятость,1.0,NaN,1100265000890,NaN,Communal,COMPANY,Вакансия службы занятости,Одобрено,f,NaN,Социальный пакет / кат.Д,"452607, г Октябрьский, ул Кувыкина, д. 3, корп. а",Республика Башкортостан,54.486407,53.483033,0,NaN,0.0,NaN,NaN,1100265000890,0,0.0,NaN,NaN,114428.0,200000000000,NaN,NaN,"Управлять автомобилем, соблюдать правила повед...",NaN,"Профессиональное вождение автомобиля, обеспечи...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Водитель автомобиля,0,Полный рабочий день,1,0.0,25000.0,2019,12
2,2,1.0,NaN,Новое рабочее место / Наружные трубопроводы и ...,от 40000 до 40000,40000.0,40000.0,NaN,0.0,2020-08-22,1,2020-08-21,NaN,2020-08-21 00:00:00,0,0.0,NaN,0,0,0,0,0,NaN,Среднее профессиональное,NaN,Полная занятость,1.0,4.0,1180280071058,NaN,Industry,COMPANY,Служба занятости,Одобрено,f,NaN,Предоставление жилья / Социальный пакет,"450076, г Уфа, ул Пушкина, д. 35, офис 2",Республика Башкортостан,54.725753,55.925139,0,NaN,0.0,NaN,NaN,1180280071058,0,0.0,NaN,NaN,145445.0,200000000000,0.0,NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Монтажник,0,Полный рабочий день,5,0.0,40000.0,2019,12


In [505]:
df_date_mistake.loc[(df_date_mistake['date_creation_mistake'] == 1) & (df_date_mistake['date_creation_month'].notna())]

,id,accommodation_capability,accommodation_housing,additional_info,base_salary,base_salary_max,base_salary_min,career_perspective,caring_workers,date_creation,date_creation_mistake,date_posted,date_posted_mistake,date_time_posted,disabled,dms,drive_licences,driver_licence_a,driver_licence_b,driver_licence_c,driver_licence_d,driver_licence_e,education_academic_degree,education_requirements_education_type,education_requirements_speciality,employment_type,experience_requirements,federal_district,id_hiring_organization,incentive_compensation_transport_compensation,industry,inner_info_contact_source,inner_info_source_type,inner_info_status,is_uzbekistan_recruitment,job_benefits,job_benefits_other_benefits,job_location_additional_address_info,job_location_address,job_location_geo_latitude,job_location_geo_longitude,large_families,metro_station,minor_workers,need_medcard,okso_code,organization,payment_meals,payment_sports_activities,premium_size,premium_type,profession,region,released_persons,requirements_id_priority_category,requirements_qualifications,requirements_required_certificates,responsibilities,retraining_condition,retraining_grant,retraining_grant_value,single_parent,social_protecteds_social_protected,source,time_posted,title,vouchers_health_institutions,work_hours,work_places,workers_with_disabled_children,mean_salary,date_creation_year,date_creation_month
0,0,NaN,NaN,Производство строительных металлических констр...,от 50000 до 50000,50000.0,50000.0,NaN,0.0,2019-07-10,1,2019-07-05,NaN,2019-07-05 00:00:00,0,0.0,NaN,0,0,0,0,0,NaN,Высшее,Металлообрабатывающие станки и комплексы,Полная занятость,1.0,NaN,1027739386938,NaN,Industry,COMPANY,Вакансия службы занятости,Одобрено,NaN,NaN,Социальный пакет / социальный пакет,"141281, Московская обл, Ивантеевка г, Санаторн...",Московская область,55.986148,37.923422,0,NaN,0.0,NaN,NaN,1027739386938,0,0.0,NaN,NaN,NaN,5000000000000,NaN,NaN,"Должен знать: постановления, распоряжения, при...",NaN,"Разрабатывает эскизные, технические и рабочие ...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Инженер-конструктор,0,Полный рабочий день,1,0.0,50000.0,2019,12
1,1,NaN,NaN,Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...,от 25000 до 25000,25000.0,25000.0,NaN,0.0,2019-12-27,1,2019-12-26,NaN,2019-12-26 00:00:00,0,0.0,[D],0,0,0,1,0,NaN,Среднее профессиональное,NaN,Полная занятость,1.0,NaN,1100265000890,NaN,Communal,COMPANY,Вакансия службы занятости,Одобрено,f,NaN,Социальный пакет / кат.Д,"452607, г Октябрьский, ул Кувыкина, д. 3, корп. а",Республика Башкортостан,54.486407,53.483033,0,NaN,0.0,NaN,NaN,1100265000890,0,0.0,NaN,NaN,114428.0,200000000000,NaN,NaN,"Управлять автомобилем, соблюдать правила повед...",NaN,"Профессиональное вождение автомобиля, обеспечи...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Водитель автомобиля,0,Полный рабочий день,1,0.0,25000.0,2019,12
2,2,1.0,NaN,Новое рабочее место / Наружные трубопроводы и ...,от 40000 до 40000,40000.0,40000.0,NaN,0.0,2020-08-22,1,2020-08-21,NaN,2020-08-21 00:00:00,0,0.0,NaN,0,0,0,0,0,NaN,Среднее профессиональное,NaN,Полная занятость,1.0,4.0,1180280071058,NaN,Industry,COMPANY,Служба занятости,Одобрено,f,NaN,Предоставление жилья / Социальный пакет,"450076, г Уфа, ул Пушкина, д. 35, офис 2",Республика Башкортостан,54.725753,55.925139,0,NaN,0.0,NaN,NaN,1180280071058,0,0.0,NaN,NaN,145445.0,200000000000,0.0,NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Монтажник,0,Полный рабочий день,5,0.0,40000.0,2019,12


In [498]:
df_date_mistake.loc[(df_date_mistake['date_creation_mistake'] == 1) & (df_date_mistake['date_creation_year'].notna())]

,id,accommodation_capability,accommodation_housing,additional_info,base_salary,base_salary_max,base_salary_min,career_perspective,caring_workers,date_creation,date_creation_mistake,date_posted,date_posted_mistake,date_time_posted,disabled,dms,drive_licences,driver_licence_a,driver_licence_b,driver_licence_c,driver_licence_d,driver_licence_e,education_academic_degree,education_requirements_education_type,education_requirements_speciality,employment_type,experience_requirements,federal_district,id_hiring_organization,incentive_compensation_transport_compensation,industry,inner_info_contact_source,inner_info_source_type,inner_info_status,is_uzbekistan_recruitment,job_benefits,job_benefits_other_benefits,job_location_additional_address_info,job_location_address,job_location_geo_latitude,job_location_geo_longitude,large_families,metro_station,minor_workers,need_medcard,okso_code,organization,payment_meals,payment_sports_activities,premium_size,premium_type,profession,region,released_persons,requirements_id_priority_category,requirements_qualifications,requirements_required_certificates,responsibilities,retraining_condition,retraining_grant,retraining_grant_value,single_parent,social_protecteds_social_protected,source,time_posted,title,vouchers_health_institutions,work_hours,work_places,workers_with_disabled_children,mean_salary,date_creation_year
0,0,NaN,NaN,Производство строительных металлических констр...,от 50000 до 50000,50000.0,50000.0,NaN,0.0,2019-07-10,1,2019-07-05,NaN,2019-07-05 00:00:00,0,0.0,NaN,0,0,0,0,0,NaN,Высшее,Металлообрабатывающие станки и комплексы,Полная занятость,1.0,NaN,1027739386938,NaN,Industry,COMPANY,Вакансия службы занятости,Одобрено,NaN,NaN,Социальный пакет / социальный пакет,"141281, Московская обл, Ивантеевка г, Санаторн...",Московская область,55.986148,37.923422,0,NaN,0.0,NaN,NaN,1027739386938,0,0.0,NaN,NaN,NaN,5000000000000,NaN,NaN,"Должен знать: постановления, распоряжения, при...",NaN,"Разрабатывает эскизные, технические и рабочие ...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Инженер-конструктор,0,Полный рабочий день,1,0.0,50000.0,2019
1,1,NaN,NaN,Новое рабочее место / ВОДИТЕЛЬ АВТОБУСА. кат.Д...,от 25000 до 25000,25000.0,25000.0,NaN,0.0,2019-12-27,1,2019-12-26,NaN,2019-12-26 00:00:00,0,0.0,[D],0,0,0,1,0,NaN,Среднее профессиональное,NaN,Полная занятость,1.0,NaN,1100265000890,NaN,Communal,COMPANY,Вакансия службы занятости,Одобрено,f,NaN,Социальный пакет / кат.Д,"452607, г Октябрьский, ул Кувыкина, д. 3, корп. а",Республика Башкортостан,54.486407,53.483033,0,NaN,0.0,NaN,NaN,1100265000890,0,0.0,NaN,NaN,114428.0,200000000000,NaN,NaN,"Управлять автомобилем, соблюдать правила повед...",NaN,"Профессиональное вождение автомобиля, обеспечи...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Водитель автомобиля,0,Полный рабочий день,1,0.0,25000.0,2019
2,2,1.0,NaN,Новое рабочее место / Наружные трубопроводы и ...,от 40000 до 40000,40000.0,40000.0,NaN,0.0,2020-08-22,1,2020-08-21,NaN,2020-08-21 00:00:00,0,0.0,NaN,0,0,0,0,0,NaN,Среднее профессиональное,NaN,Полная занятость,1.0,4.0,1180280071058,NaN,Industry,COMPANY,Служба занятости,Одобрено,f,NaN,Предоставление жилья / Социальный пакет,"450076, г Уфа, ул Пушкина, д. 35, офис 2",Республика Башкортостан,54.725753,55.925139,0,NaN,0.0,NaN,NaN,1180280071058,0,0.0,NaN,NaN,145445.0,200000000000,0.0,NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,"Укладка трубопроводов НВК Ду57-1020 мм, монтаж...",NaN,0.0,NaN,0.0,NaN,EMPLOYMENT_SERVICE,00:00:00,Монтажник,0,Полный рабочий день,5,0.0,40000.0,2019


### Метод отсечения выбросов

In [68]:
def outlier_row_mean(df, row, value):
    '''Метод отсечения выбросов - выброс больше value усредняется'''
    df.loc[df[row] > value, row] = round(df[row].mean())
    return

In [69]:
#  пример использования метода outlier_row
outlier_row_mean(df_tr, 'experience_requirements', 20)

In [70]:
df_tr['experience_requirements']

16     NaN
89     3.0
111    0.0
112    1.0
115    0.0
Name: experience_requirements, dtype: float64

In [71]:
def outlier_row_zero(df, row, value):
    '''Метод отсечения выбросов - выброс больше value = 0'''
    df.loc[df[row] > value, row] = 0
    return

In [72]:
#  пример использования метода outlier_row
outlier_row_zero(df_tr, 'federal_district', 10)

In [98]:
# len(df_train['id_hiring_organization'].unique())

172858

### Метод дамми f = 0 t = 1

In [73]:
def dummy_ft(df, row):
    df.loc[df[row] == 'f', row] = 0
    df.loc[df[row] == 't', row] = 1
    return

In [74]:
#  применение метода category_change через метод работы с полями
rows_list = ['is_uzbekistan_recruitment']
row_worker(df_tr, rows_list, dummy_ft)

In [75]:
df_tr['is_uzbekistan_recruitment']

16     NaN
89     NaN
111      0
112      0
115    NaN
Name: is_uzbekistan_recruitment, dtype: object

### Корректировка полей по премии

In [76]:
df_tr = df_train.loc[(df_train['premium_type'] == 'Ежемесячная премия') \
             | (df_train['premium_type'] == 'Ежеквартальная премия')\
             | (df_train['premium_type'] == 'Ежегодная премия')].head(50)

In [331]:
# df_tr['premium_type']

id
119        Ежемесячная премия
229     Ежеквартальная премия
565        Ежемесячная премия
716        Ежемесячная премия
1027       Ежемесячная премия
Name: premium_type, dtype: object

In [77]:
# Корректируем ошибочные данные.
# Не может быть премия 10 руб., вероятно это 10 тыс.
df_tr.loc[(df_tr['premium_size'] < 1000), 'premium_size'] = df_tr['premium_size'] * 1000

In [78]:
# Переводим тип премии в количество месяцев
df_tr.loc[(df_tr['premium_type'] == 'Ежемесячная премия'), 'premium_type'] = 1
df_tr.loc[df_tr['premium_type'] == 'Ежеквартальная премия', 'premium_type'] = 4
df_tr.loc[df_tr['premium_type'] == 'Ежегодная премия', 'premium_type'] = 12

In [79]:
# генерим новое поле ежемесячной премии
df_tr['premium_size_month'] = df_tr['premium_size'] / df_tr['premium_type']

In [80]:
# удаляем ненужные для модели поля после их обработки
df_tr.drop(['premium_size', 'premium_type'], axis = 1, inplace = True)

In [319]:
print(df_train['premium_type'].value_counts())
print('Заполнены', df_train['premium_type'].loc[df_train['premium_type'].notna()].count())


Ежемесячная премия       3643
Ежеквартальная премия     732
Ежегодная премия          536
Name: premium_type, dtype: int64
Заполнены 4911


### Метод работы с выбросами < value для retraining_grant_value

In [386]:
def outlier_row_zero_less(df, row, value):
    '''Метод отсечения выбросов - выброс меньше value = 0'''
    df.loc[df[row] < value, row] = 0
    return

In [387]:
outlier_row_zero_less(df_tr, 'retraining_grant_value', 5000)

### Метод

In [81]:
# удаляем ненужные поля
df_tr.drop(['time_posted'], axis = 1, inplace = True)

In [82]:
# присваиваем nan заданное значение
def nan_value(df, row, value):
    '''метод присвоения nan заданного значения'''
    df.loc[df[row].isna(), row] = value
    return

In [83]:
# заполняем Nan заданными значениями
nan_value(df_tr, 'work_hours', '«Полный рабочий день»')
nan_value(df_tr, 'work_places', 1)

### Заполняем пустые значения во всём датафрейме

In [84]:
df_tr = df_tr.fillna(0)